In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import pickle
import numpy as np

# Load datasets
# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_data = df
emg_data = pd.read_csv('EMGF1.csv')

# Assuming the last column is the label for both datasets
X_eog = eog_data.iloc[:, :-1].values
y_eog = eog_data.iloc[:, -1].values

X_emg = emg_data.iloc[:, :-1].values
y_emg = emg_data.iloc[:, -1].values

# Split into train and test sets
X_eog_train, X_eog_test, y_eog_train, y_eog_test = train_test_split(X_eog, y_eog, test_size=0.1, random_state=42)
X_emg_train, X_emg_test, y_emg_train, y_emg_test = train_test_split(X_emg, y_emg, test_size=0.2, random_state=42)

# Save the test labels
y_test_combined = np.concatenate((y_eog_test, y_emg_test))
np.save('y_test_combined.npy', y_test_combined)


**Ensemble and test**

In [32]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

# Function to load models
def load_models():
    model_EOG = joblib.load('EOG_ensemble_model.sav')
    model_EMG = joblib.load('EMG_ensemble_model.sav')
    tsne_eog = joblib.load('tsne_eog.sav')
    tsne_emg = joblib.load('tsne_emg.sav')
    return model_EOG, model_EMG, tsne_eog, tsne_emg

# Function to preprocess input using t-SNE
def preprocess_with_tsne(data, tsne_model):
    return tsne_model.fit_transform(data)

# Function to downsample data
def downsample_data(data, target_size):
    return resample(data, n_samples=target_size, random_state=42)

# Function to classify input
def classify_input(X_input_eog, X_input_emg, y_test_combined):
    # Load models and t-SNE
    model_EOG, model_EMG, tsne_eog, tsne_emg = load_models()

    # Downsample the input EMG data to match the size of EOG data
    X_input_emg_downsampled = downsample_data(X_input_emg, len(X_input_eog))

    # Preprocess the input using t-SNE
    X_input_eog_tsne = preprocess_with_tsne(X_input_eog, tsne_eog)
    X_input_emg_tsne = preprocess_with_tsne(X_input_emg_downsampled, tsne_emg)

    # Predict using each model
    pred_EOG = model_EOG.predict(X_input_eog_tsne)
    pred_EMG = model_EMG.predict(X_input_emg_tsne)

    # Create meta-features by stacking the predictions
    meta_features = np.column_stack((pred_EOG, pred_EMG))

    # Train a meta-classifier
    meta_classifier = RandomForestClassifier(random_state=42)
    meta_classifier.fit(meta_features, y_test_combined[:len(meta_features)])  # Use only matching size of y_test_combined

    # Save the meta-classifier
    with open('meta_classifier.sav', 'wb') as file:
        pickle.dump(meta_classifier, file)

    # Predict using the meta-classifier
    final_predictions = meta_classifier.predict(meta_features)

    return final_predictions

# Example usage
# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_test_data = df.values
emg_test_data = pd.read_csv('EMGF1.csv').values
y_test_combined = np.load('y_test_combined.npy')

predictions = classify_input(eog_test_data, emg_test_data, y_test_combined)

# Evaluate the performance
accuracy = accuracy_score(y_test_combined[:len(predictions)], predictions)
print(f'Combined Model Accuracy: {accuracy}')



/home/love/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/love/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/love/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.4.1.post1 w

Combined Model Accuracy: 0.6293726510552182


**With new dataset**

**Saving Labels**

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import pickle
import numpy as np

# Load datasets
# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_data = df
emg_data = pd.read_csv('EMGF1.csv')

# Assuming the last column is the label for both datasets
X_eog = eog_data.iloc[:, :-1].values
y_eog = eog_data.iloc[:, -1].values

X_emg = emg_data.iloc[:, :-1].values
y_emg = emg_data.iloc[:, -1].values

# Split into train and test sets
X_eog_train, X_eog_test, y_eog_train, y_eog_test = train_test_split(X_eog, y_eog, test_size=0.1, random_state=42)
X_emg_train, X_emg_test, y_emg_train, y_emg_test = train_test_split(X_emg, y_emg, test_size=0.2, random_state=42)

# Save the test labels
y_test_combined = np.concatenate((y_eog_test, y_emg_test))
np.save('y_test_combined.npy', y_test_combined)


**Saving the preprocessed data**

In [23]:
from sklearn.manifold import TSNE
import joblib

# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_data = df.values
emg_data = pd.read_csv('EMGF1.csv').values

# Fit the t-SNE models
tsne_eog = TSNE(n_components=3, random_state=42)
tsne_eog.fit(eog_data)

tsne_emg = TSNE(n_components=3, random_state=42)
tsne_emg.fit(emg_data)

# Save the t-SNE models
joblib.dump(tsne_eog, 'tsne_eog.sav')
joblib.dump(tsne_emg, 'tsne_emg.sav')


['tsne_emg.sav']

In [25]:
joblib.dump(tsne_eog, 'tsne_eog.sav')

['tsne_eog.sav']

**Using SVC**

In [29]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import pickle
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Function to load models
def load_models():
    model_EOG = joblib.load('eog_random_forest_model.sav')
    model_EMG = joblib.load('random_forest_model.sav')
    tsne_eog = joblib.load('tsne_eog.sav')
    tsne_emg = joblib.load('tsne_emg.sav')
    return model_EOG, model_EMG, tsne_eog, tsne_emg

# Function to preprocess input using t-SNE
def preprocess_with_tsne(data, tsne_model):
    return tsne_model.fit_transform(data)

# Function to downsample data
def downsample_data(data, target_size):
    return resample(data, n_samples=target_size, random_state=42)

# Function to classify input
def classify_input(X_input_eog, X_input_emg, y_test_combined):
    # Load models and t-SNE
    model_EOG, model_EMG, tsne_eog, tsne_emg = load_models()

    # Downsample the input EMG data to match the size of EOG data
    X_input_emg_downsampled = downsample_data(X_input_emg, len(X_input_eog))

    # Preprocess the input using t-SNE
    X_input_eog_tsne = preprocess_with_tsne(X_input_eog, tsne_eog)
    X_input_emg_tsne = preprocess_with_tsne(X_input_emg_downsampled, tsne_emg)

    # Predict using each model
    pred_EOG = model_EOG.predict(X_input_eog_tsne)
    pred_EMG = model_EMG.predict(X_input_emg_tsne)

    # Create meta-features by stacking the predictions
    meta_features = np.column_stack((pred_EOG, pred_EMG))

    # Train a meta-classifier
    meta_classifier =SVC()
    meta_classifier.fit(meta_features, y_test_combined[:len(meta_features)])  # Use only matching size of y_test_combined

    # Save the meta-classifier
    with open('meta_classifier.sav', 'wb') as file:
        pickle.dump(meta_classifier, file)

    # Predict using the meta-classifier
    final_predictions = meta_classifier.predict(meta_features)

    return final_predictions

# Example usage
# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_test_data = df.values

emg_test_data = pd.read_csv('EMGF1.csv').values
y_test_combined = np.load('y_test_combined.npy')

predictions = classify_input(eog_test_data, emg_test_data, y_test_combined)

# Evaluate the performance
accuracy = accuracy_score(y_test_combined[:len(predictions)], predictions)
print(f'Combined Model Accuracy: {accuracy}')




Combined Model Accuracy: 0.6293726510552182


**Using RandomForest**

In [28]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import pickle
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Function to load models
def load_models():
    model_EOG = joblib.load('eog_random_forest_model.sav')
    model_EMG = joblib.load('random_forest_model.sav')
    tsne_eog = joblib.load('tsne_eog.sav')
    tsne_emg = joblib.load('tsne_emg.sav')
    return model_EOG, model_EMG, tsne_eog, tsne_emg

# Function to preprocess input using t-SNE
def preprocess_with_tsne(data, tsne_model):
    return tsne_model.fit_transform(data)

# Function to downsample data
def downsample_data(data, target_size):
    return resample(data, n_samples=target_size, random_state=42)

# Function to classify input
def classify_input(X_input_eog, X_input_emg, y_test_combined):
    # Load models and t-SNE
    model_EOG, model_EMG, tsne_eog, tsne_emg = load_models()

    # Downsample the input EMG data to match the size of EOG data
    X_input_emg_downsampled = downsample_data(X_input_emg, len(X_input_eog))

    # Preprocess the input using t-SNE
    X_input_eog_tsne = preprocess_with_tsne(X_input_eog, tsne_eog)
    X_input_emg_tsne = preprocess_with_tsne(X_input_emg_downsampled, tsne_emg)

    # Predict using each model
    pred_EOG = model_EOG.predict(X_input_eog_tsne)
    pred_EMG = model_EMG.predict(X_input_emg_tsne)

    # Create meta-features by stacking the predictions
    meta_features = np.column_stack((pred_EOG, pred_EMG))

    # Train a meta-classifier
    meta_classifier =RandomForestClassifier(random_state=42)
    meta_classifier.fit(meta_features, y_test_combined[:len(meta_features)])  # Use only matching size of y_test_combined

    # Save the meta-classifier
    with open('meta_classifier.sav', 'wb') as file:
        pickle.dump(meta_classifier, file)

    # Predict using the meta-classifier
    final_predictions = meta_classifier.predict(meta_features)

    return final_predictions

# Example usage
# Assuming you have the EOG and EMG data
df=pd.read_excel('EOGsignal.xlsx').iloc[:, 1:]
# Rename the columns
new_column_names = ['f' + str(i) for i in range(0, 24)] + ['class']
df.columns = new_column_names

eog_test_data = df.values

emg_test_data = pd.read_csv('EMGF1.csv').values
y_test_combined = np.load('y_test_combined.npy')

predictions = classify_input(eog_test_data, emg_test_data, y_test_combined)

# Evaluate the performance
accuracy = accuracy_score(y_test_combined[:len(predictions)], predictions)
print(f'Combined Model Accuracy: {accuracy}')



Combined Model Accuracy: 0.6293726510552182
